In [23]:
import mysql.connector
import random
from datetime import datetime, timedelta

connection = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

cursor = connection.cursor()

# Pobranie najwyższego istniejącego id_rezerwacje z bazy danych
cursor.execute("SELECT COALESCE(MAX(id_rezerwacje), 0) FROM Rezerwacje")
max_id_rezerwacje = cursor.fetchone()[0]

insert_data_query = """
INSERT INTO Rezerwacje (id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, Data_rezerwacji)
VALUES (%s, %s, %s, %s, %s, %s);
"""

FIRMA_START_DATE = datetime(2022, 1, 14).date()  # Zamiana na date

def losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia):
    max_date = data_rozpoczecia - timedelta(days=2)
    min_date = max(FIRMA_START_DATE, data_urodzenia + timedelta(weeks=52*18), data_rozpoczecia - timedelta(weeks=26))
    if min_date > max_date:
        min_date = max_date  # W przypadku braku odpowiedniego zakresu, data minimalna równa się maksymalnej
    delta = max_date - min_date
    random_days = random.randint(0, delta.days)
    return min_date + timedelta(days=random_days)


# Pobieranie danych z bazy danych
cursor.execute("SELECT id_wycieczki, Data_rozpoczecia FROM Realizowane_wycieczki")
wycieczki = cursor.fetchall()

cursor.execute("SELECT id_klienci, Data_urodzenia FROM Klienci")
klienci = cursor.fetchall()

cursor.execute("SELECT id_pracownicy FROM Pracownicy")
pracownicy = cursor.fetchall()

cursor.execute("SELECT id_transakcje FROM Transakcje_finansowe")
transakcje = [transakcja[0] for transakcja in cursor.fetchall()]

# Przygotowanie mapy klient -> transakcja
transakcje_klienci = {}
for klient, data_urodzenia in klienci:
    if not transakcje:
        raise ValueError("Za mało transakcji, aby przypisać unikalną każdemu klientowi.")
    transakcje_klienci[klient] = transakcje.pop(0)

rezerwacje = []

# Tworzenie rezerwacji dla każdego klienta (przynajmniej jedna rezerwacja na klienta)
for klient, data_urodzenia in klienci:
    id_klienci = klient
    id_transakcje = transakcje_klienci[id_klienci]

    # Losowanie wycieczki i daty rezerwacji dopóki wiek nie osiągnie 18 lat
    while True:
        id_wycieczki, data_rozpoczecia = random.choice(wycieczki)
        data_rezerwacji = losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia)
        wiek_klienta = (data_rezerwacji - data_urodzenia).days // 365
        if wiek_klienta >= 18:
            break

    id_pracownicy = random.choice([pracownik[0] for pracownik in pracownicy if 5 <= pracownik[0] <= 7])

    max_id_rezerwacje += 1
    rezerwacje.append((max_id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, data_rezerwacji.strftime('%Y-%m-%d')))

# Sortowanie rezerwacji według daty rezerwacji, a następnie ponowne przypisanie id_rezerwacje
rezerwacje.sort(key=lambda x: datetime.strptime(x[5], '%Y-%m-%d'))
for i in range(len(rezerwacje)):
    rezerwacje[i] = (i + 1, *rezerwacje[i][1:])

# Tworzenie dodatkowych rezerwacji do osiągnięcia 500 wpisów
while len(rezerwacje) < 500:
    id_klienci, data_urodzenia = random.choice(klienci)
    id_wycieczki, data_rozpoczecia = random.choice(wycieczki)
    data_rezerwacji = losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia)

    wiek_klienta = (data_rezerwacji - data_urodzenia).days // 365
    if wiek_klienta < 18:
        continue

    if id_klienci in transakcje_klienci:
        id_transakcje = transakcje_klienci[id_klienci]
    else:
        id_transakcje = random.choice(transakcje)

    id_pracownicy = random.choice([pracownik[0] for pracownik in pracownicy if 5 <= pracownik[0] <= 7])

    max_id_rezerwacje += 1
    rezerwacje.append((max_id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, data_rezerwacji.strftime('%Y-%m-%d')))

# Sortowanie ostatecznej listy rezerwacji według daty rezerwacji i przypisanie finalnych id_rezerwacje
rezerwacje.sort(key=lambda x: datetime.strptime(x[5], '%Y-%m-%d'))
for i in range(len(rezerwacje)):
    rezerwacje[i] = (i + 1, *rezerwacje[i][1:])

# Wstawianie danych do bazy
cursor.executemany(insert_data_query, rezerwacje)
connection.commit()

cursor.close()
connection.close()

print("Rezerwacje zostały pomyślnie wstawione do bazy danych.")



Rezerwacje zostały pomyślnie wstawione do bazy danych.
